In [1]:
# 초기 설치해야하는 라이브러리
# 초기 1회만 설치하면 됨

!pip install pd
!pip install selenium
!pip install requests

You should consider upgrading via the 'C:\Users\82102\Anaconda3\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\82102\Anaconda3\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\82102\Anaconda3\python.exe -m pip install --upgrade pip' command.


In [90]:
!pip install multiprocessing

     -------------------------------------- 108.0/108.0 KB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [7 lines of output]
  Traceback (most recent call last):
    File "<string>", line 2, in <module>
    File "<pip-setuptools-caller>", line 34, in <module>
    File "C:\Users\82102\AppData\Local\Temp\pip-install-9qt_y_u6\multiprocessing_b6825f3ee8f74d1aa1070970bc941500\setup.py", line 94
      print 'Macros:'
            ^
  SyntaxError: Missing parentheses in call to 'print'. Did you mean print('Macros:')?
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

Encountered error while generating package metadata.

See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
You should consider upgrading via the 'C:\Users\82102\Anaconda3\python.exe -m pip install --upgrade pip' command.


In [83]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from requests import get
import requests
from urllib import parse
from urllib import request
import os
import asyncio
from bs4 import BeautifulSoup as bs 
from multiprocessing import Process, Queue

In [6]:
# 순차
def GetData(filename, key): 
    # 엑셀에서 데이터 받아오는 함수
    # filename에 엑셀파일 경로, key에 column 의 이름을 기입하면 해당 열을 반환 
    # filename의 \는 반드시 \\ 또는 /으로 바꿔써야한다.
    # key는 소문자로 입력해도 상관없음
    
    key = key.upper()
    data = pd.read_excel(filename , engine = "openpyxl")
    
    if(key in data.columns):
        return data[key]
    else:
        print("Key is not defined")
        
##################################################################################

def DownloadPdf_req(pdf_name, pmid):
    #requests 라이브러리 사용
    # pdf_name 에 입력한대로 저장된다
    # pmid 에 PMID값 넣어서 다운로드
    
    #headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'}
    a = requests.post('https://sci-hub.se/', data = {'request': pmid})
    r = bs(a.text, 'html.parser')
    
    try: # 논문 없을때
        try:  # 디도스 가드
            c = r.select_one('div > button').attrs['onclick'] 
            c = c[15:-1]
            if('sci-hub.se' in c):
                c = 'https:' + c
            else:
                c = 'https://sci-hub.se' + c
                opener = request.build_opener()
                opener.addheaders = [('User-agent', 'Mozilla/5.0')]
                request.install_opener(opener)
        except:
            data_frame = [pdf_name, pmid]
            print('논문이 없습니다.')
            print()
            return data_frame
        
        current_path = str(os.getcwd()) + "/다운로드된 논문/" + pdf_name
        try:
            request.urlretrieve(c, current_path)  # 다운로드
        except:
            r = requests.get(c)
            with open(current_path, 'wb') as outfile:
                outfile.write(r.content)
        print(pdf_name + ' 다운로드 완료')
        print()
    except:
        print(pdf_name+ ' 다운로드 실패')
        data_frame = [pdf_name, pmid]
        print()
        return data_frame
        
        
##################################################################################

def DownloadPdf_sel(pdf_name, pmid):
    # selenium 라이브러리 사용
    # pdf_name 에 입력한대로 저장된다
    # pmid 에 PMID값 넣어서 다운로드
    
    browser = webdriver.Chrome()
    browser.get("https://sci-hub.se/")

    search_area = browser.find_element_by_id("request") # 검색창 
    search_area.send_keys(pmid)
    search_area.send_keys(Keys.ENTER)
    
    try:
        pdf_url = browser.find_element_by_id("pdf").get_attribute("src") # pdf url
        browser.get(pdf_url)

        current_path = str(os.getcwd()) + "/다운로드된 논문/" + pdf_name


        with open(current_path, "wb") as file:
            response = get(pdf_url)
            file.write(response.content)
    except:
        print("에러 발생" + "_" + pdf_name)
    
##################################################################################

def DownloadJournal(filename, type_):
    df_pmid = GetData(filename, 'pmid')
    df_title = GetData(filename, 'title')
    length = len(df_pmid)
    data_ = []
    
    try:
        os.mkdir("다운로드된 논문")
    except:
        print('')
    
    if(type_ == 'sel'):
        for i in range(0, length):
            title = str(i+1) + "_" + df_title[i][0:30] + ".pdf"
            pmid = int(df_pmid[i])

            DownloadPdf_sel(title, pmid)
    
    elif(type_ == 'req'):     
        for i in range(0, length):
            title = str(i+1) + "_" + df_title[i][0:30] + ".pdf"
            pmid = int(df_pmid[i])

            data_frame = DownloadPdf_req(title, pmid)
            if(data_frame != None):
                data_.append(data_frame)
                
        df_ = pd.DataFrame(data_,columns=['num+title', 'pmid'])
        print(df_)
        df_.to_csv("./다운로드된 논문/undownloaded_journal.csv",mode='w')

In [7]:
## filename에 경로 넣고 경로사이 구분은 \\로, req로 하는게 빠르다!
## 그냥 filename만 바꾸기!

filename ='C:\\Users\\82102\\아나콘다\\논문 다운로드\엑셀.xlsx'

DownloadJournal(filename, 'req')


1_Bovine dairy complex lipids im.pdf 다운로드 완료

2_The kiwi fruit peptide kissper.pdf 다운로드 완료

3_Effect of garlic sulfur compou.pdf 다운로드 완료

논문이 없습니다.

5_Effects of Puerariae radix ext.pdf 다운로드 완료

6_A Novel Non-Digestible, Carrot.pdf 다운로드 완료

7_The impact of tomato fruits co.pdf 다운로드 완료

8_Plant phenolics decrease intes.pdf 다운로드 완료

                              num+title      pmid
0  4_Effects of dietary asafoetida .pdf  32681211


In [86]:
# 비동기
def GetData_(filename, key): 
    # 엑셀에서 데이터 받아오는 함수
    # filename에 엑셀파일 경로, key에 column 의 이름을 기입하면 해당 열을 반환 
    # filename의 \는 반드시 \\ 또는 /으로 바꿔써야한다.
    # key는 소문자로 입력해도 상관없음
    
    key = key.upper()
    data = pd.read_excel(filename , engine = "openpyxl")
    
    if(key in data.columns):
        return data[key]
    else:
        print("Key is not defined")
        
##################################################################################

def DownloadPdf_req_(pdf_name, pmid):
    #requests 라이브러리 사용
    # pdf_name 에 입력한대로 저장된다
    # pmid 에 PMID값 넣어서 다운로드
    
    #headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'}
    a = requests.post('https://sci-hub.se/', data = {'request': pmid})
    r = bs(a.text, 'html.parser')
    
    try: # 논문 없을때
        try:  # 디도스 가드
            c = r.select_one('div > button').attrs['onclick'] 
            c = c[15:-1]
            if('sci-hub.se' in c):
                c = 'https:' + c
            else:
                c = 'https://sci-hub.se' + c
                opener = request.build_opener()
                opener.addheaders = [('User-agent', 'Mozilla/5.0')]
                request.install_opener(opener)
        except:
            data_frame = [pdf_name, pmid]
            print('논문이 없습니다.')
            print()
            return data_frame
        
        current_path = str(os.getcwd()) + "/다운로드된 논문/" + pdf_name
        try:
            request.urlretrieve(c, current_path)  # 다운로드
        except:
            r = requests.get(c)
            with open(current_path, 'wb') as outfile:
                outfile.write(r.content)
        print(pdf_name + ' 다운로드 완료')
        print()
    except:
        print(pdf_name+ ' 다운로드 실패')
        data_frame = [pdf_name, pmid]
        print()
        return data_frame
        
        
##################################################################################

def DPMultiProcessing(start, end, df_title, df_pmid, data_):
    for i in range(start, end):
        title = str(i+1) + "_" + df_title[i][0:30] + ".pdf" # 이름
        pmid = int(df_pmid[i]) # pmid

        data_frame = DownloadPdf_req_(title, pmid)
        if(data_frame != None):
            data_.append(data_frame)
    print(start, 바보) ##########

##################################################################################

def DownloadJournal_mp(filename, processing_): # processing은 반드시 1이상
    df_pmid = GetData_(filename, 'pmid')
    df_title = GetData_(filename, 'title')
    length = len(df_pmid)
    data_ = []
    
    try:
        os.mkdir("다운로드된 논문")
    except:
        print('')
    
    num_ = []
    for ij in range(0, processing_ -1):
        num_.append(length//processing_*(ij+1))
    num_.append(length)
    
    st = 0
    en = 0
    for i in range(0, processing_):
        st = en
        en = num_[i]
        #globals()['p'+str(i)] = Process(target=DPMultiProcessing, args=(st, en, df_title, df_pmid, data_))
        globals()['p'+str(i)] = Process(target = mp, args=(str(i)))
    for j in range(0, processing_):
        globals()['p'+str(j)].start(); globals()['p'+str(j)].join()
    
    df_ = pd.DataFrame(data_,columns=['num+title', 'pmid'])
    print(df_)
    df_.to_csv("./다운로드된 논문/undownloaded_journal.csv",mode='w')

In [88]:
###
def mp():
    print('밥')

pp1 = Process(target = mp)
pp1.start()
pp1.join()

In [84]:
filename ='C:\\Users\\82102\\아나콘다\\논문 다운로드\엑셀 - 복사본.xlsx'

DownloadJournal_mp(filename, 4)


Empty DataFrame
Columns: [num+title, pmid]
Index: []


In [16]:
for ii in range(0,3):
    globals()['hi'+str(ii)] = ii+3

In [17]:
for jj in range(0,3):
    print(globals()['hi'+str(jj)])

3
4
5
